## We require four subsets for the regression analysis
- All data
- Countries with above average trade with UK
- OECD countries
- EU countries

In [1]:
import pandas as pd

df = pd.read_csv('./data/cleaned.csv')

In [2]:
df1 = df.copy()

In [3]:
#We define a descending list of average trade with UK and use it to get subset 2
import math

tradeflows = df[df['iso3_d'] == 'GBR'].groupby('iso3_o').mean()['ltradeflow'].sort_values(ascending=False)
above_average = list(tradeflows[0:math.floor(len(tradeflows)/2)].index)

df2 = df[(df['iso3_d'] == 'GBR') & df['iso3_o'].isin(above_average)].copy()

In [4]:
#Creating subset 3, using a list of all OECD countries (with iso codes)
OECD = ['AUS', 'AUT', 'BEL', 'CAN','CHE', 'CHL', 'COL', 'CRI', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HUN', 'IRL', 'ISL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SVN', 'SWE', 'TUR', 'USA']

df3 = df[df['iso3_o'].isin(OECD) & df['iso3_d'].isin(OECD)].copy()

In [5]:
#Creating subset 4
df4 = df[df['both_eu'] == 1].copy()

## De-meaning for fixed effects regression
- In the fixed effects model, we include importer and exporter dummies for each time period
- There will be approximately 20,000 dummies if estimated directly, so use double de-meaning instead
- We de-mean in the exporter (origin) dimension first, and then the importer (destination) dimension

In [6]:
df_fe = pd.read_csv('./data/cleaned_fe.csv')

In [7]:
def demean(df, fe1, fe2):
    data = df.copy()
    
    data['fe1'] = ''
    data['fe2'] = ''
    for column in fe1: 
        data['fe1'] += data[column].astype('str')
    for column in fe2:
        data['fe2'] += data[column].astype('str')
    
#     demeaner1 = data.drop('fe2', axis=1).groupby('fe1').mean()
#     demeaner1 = data[['fe1']].join(demeaner1, on='fe1')\
#                 .drop(['fe1'], axis=1)
#     demeaner2 = data.drop('fe1', axis=1).groupby('fe2').mean()
#     demeaner2 = data[['fe2']].join(demeaner2, on='fe2')\
#             .drop(['fe2'], axis=1)
#     data = data.drop(['fe1', 'fe2'], axis=1)
#     result = data - demeaner1 - demeaner2 + data.mean()
#     result[fe1] = df[fe1]
#     result[fe2] = df[fe2]
#     return result

demeaned = demean(df_fe, ['iso3_o','year'], ['iso3_d','year'])


## Visualising residuals
- We aim to visualise residuals from different regression specifications to highlight the types of errors
- The UK is chosen to be the reference country

In [8]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import geopandas as gpd
from geopandas import GeoDataFrame

In [9]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

### Defining the visualisation function
- The ```animate()``` function takes the name of a reference country and a dataframe containing a column named ```prediction```
- It then uses plotly to generate a choropleth map animated over time

In [10]:
#Takes the ISO code of country and a dataframe containing predicted values under a column named 'prediction'
def animate(country, predicted):
    actual = predicted[predicted['iso3_d'] == country][['year','iso3_o','ltradeflow']]
    predictions = predicted[predicted['iso3_d'] == country][['year', 'iso3_o', 'prediction']]
    
    merged = pd.merge(predictions,actual,on=['year','iso3_o'],how='left')
    merged = merged.drop(merged[merged['ltradeflow'].isna()].index)
    merged['error'] = merged['prediction']-merged['ltradeflow']
    merged = merged.sort_values('year', ascending=True)
    fig = px.choropleth_mapbox(merged,
        geojson=world,
        animation_frame='year',
        featureidkey='properties.iso_a3',
        locations='iso3_o',
        center={'lat':50, 'lon':0},
        color='error',
        range_color=[merged['error'].min(), merged['error'].max()],
        width=950,
        height=600,
        color_continuous_scale=['darkred', 'lime'],
        mapbox_style='carto-positron',
        zoom=2)
    fig.show()


## Regression function (OLS)
- We define a ```run_reg_ols``` function which takes three dataframes: the outcome variable, the regressors, and the entire data matrix.
- This function returns a dataframe containing a 'prediction' column, so it can be chained with animation

In [11]:
import statsmodels.api as sm
import numpy as np

#Regression function for OLS
def run_reg_ols(y,X,full):
    model = sm.OLS(y,sm.add_constant(X))
    results = model.fit()
    predicted_values = results.fittedvalues
    
    full['prediction'] = predicted_values
    
    return full

df1 spec1 (OLS)

df4 spec1 (OLS)

df1 spec2 (OLS)

df4 spec2(OLS)

df1 spec3 (OLS)

df1 spec4 (OLS)

df1 spec5 (OLS)

df4 spec5 (OLS)

## Regression function (PPML)
- We define a ```run_reg_ppml``` function which takes three dataframes: the outcome variable, the regressors, and the entire data matrix.
- This function returns a dataframe containing a 'prediction' column, so it can be chained with animation

In [12]:
#Regression function for PPML
def run_reg_ppml():
    pass

df1 spec1 (PPML)

df4 spec1 (PPML)

df1 spec5 (PPML)

df4 spec5 (PPML)